In [113]:
data Category = Electronics | Books | Clothing | Groceries deriving (Show, Eq)

data Product = Product {pid :: Int, pname :: String, price :: Float, category :: Category}
instance Show Product where
    show (Product id name price category) = "Product id: " ++ show id ++ ", product: " ++ show name ++ ", price: " ++ show price ++ ", category: " ++ show category

In [114]:
data LoyaltyLevel = Bronze | Silver | Gold deriving (Show, Eq)

data Customer = Customer {cid :: Int, cname :: String, loyaltyLevel :: LoyaltyLevel}

instance Show Customer where
    show (Customer id name loyaltyLevel) = "Customer id: " ++ show id ++ ", customer: " ++ show name ++ ", loyalty level: " ++ show loyaltyLevel

In [115]:
data CartItem = CartItem {product :: Product, quantity :: Int}

instance Show CartItem where
    show (CartItem product quantity) = "Item: " ++ show product ++ ", units: " ++ show quantity

In [116]:
newtype ShoppingCart = ShoppingCart [CartItem]

instance Show ShoppingCart where
    show (ShoppingCart items) = "Shopping cart:\n" ++ unlines (map (\x -> "\t" ++ show x) items)

In [117]:
newtype Stock = Stock [(Product, Int)] -- product and their available quantity

instance Show Stock where
    show (Stock items) = "Stock:\n" ++ unlines (map (\(p, i) -> "\t" ++ show p ++ ", quantity available: " ++ show i) items)

data Status = Pending | Processing | Shipped | Delivered | Cancelled deriving Show

In [118]:
data Order = Order {customer :: Customer, shoppingCart :: ShoppingCart, totalPrice :: Float, status :: Status}

instance Show Order where
    show (Order customer shoppingCart totalPrice status) = "Order for customer: " ++ show customer ++ "\n" ++ show shoppingCart 
        ++ "Total price: " ++ show totalPrice ++ ", status: " ++ show status ++ "\n"


data SearchCriterion = ById Int | ByLoyaltyLevel LoyaltyLevel | ByProductId Int | ByCategory Category | ByTotalPrice Float deriving Show

In [119]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5

cart = ShoppingCart [cart1,  cart2]
cart

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

stock

customer = Customer 1 "Ivan" Gold
order = Order customer cart 17 Pending

order

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

Stock:
	Product id: 1, product: "Ivan", price: 12.0, category: Electronics, quantity available: 2
	Product id: 2, product: "Isa", price: 15.0, category: Clothing, quantity available: 3
	Product id: 3, product: "Siro", price: 5.0, category: Groceries, quantity available: 4

Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending

In [120]:
calculateProductPrice :: Product -> Float
calculateProductPrice (Product _ _ p _) = p

In [121]:
calculateProductPrice product1

12.0

In [122]:
instance Eq Product where
        (Product pid1 _ _ _) == (Product pid2 _ _ _) = pid1 == pid2

In [123]:
add :: CartItem -> ShoppingCart -> ShoppingCart
add item (ShoppingCart xs) = ShoppingCart (item:xs)

add cart1 cart

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

In [124]:
addToCart :: CartItem -> ShoppingCart -> ShoppingCart
addToCart item (ShoppingCart []) = ShoppingCart [item]
addToCart i@(CartItem product quantity) (ShoppingCart ((CartItem p q): cartList)) 
                | product == p = ShoppingCart (CartItem p (q+quantity) : cartList)
                | otherwise = add (CartItem p q) (addToCart i (ShoppingCart cartList))

cart

addToCart cart1 cart

addToCart (CartItem product3 12) cart

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 4
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 12

In [ ]:
isProductByIdInCart :: Int -> ShoppingCart -> Bool
isProductByIdInCart _ (ShoppingCart []) = False
isProductByIdInCart productId (ShoppingCart ((CartItem (Product pid _ _ _) _):xs)) 
        | productId == pid = True
        | otherwise = isProductByIdInCart productId (ShoppingCart xs)


isProductByCategoryInCart :: Category -> ShoppingCart -> Bool
isProductByCategoryInCart _ (ShoppingCart []) = False
isProductByCategoryInCart category (ShoppingCart ((CartItem (Product _ _ _ c) _):xs)) 
        | category == c = True
        | otherwise = isProductByCategoryInCart category (ShoppingCart xs)



searchOrders :: [SearchCriterion] -> [Order] -> [Order]
searchOrders [] orders = orders
searchOrders ((ById id): xs) orders = searchOrders xs (filter (\(Order (Customer cid _ _) _ _ _) -> cid == id) orders)
searchOrders ((ByLoyaltyLevel loyaltyLevel): xs) orders = searchOrders xs (filter (\(Order (Customer _ _ ll) _ _ _) -> loyaltyLevel == ll) orders)
searchOrders ((ByProductId id): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByIdInCart id cart) orders)
searchOrders ((ByCategory category): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByCategoryInCart category cart) orders)
searchOrders ((ByTotalPrice price): xs) orders = searchOrders xs (filter (\(Order _ _ p _) -> price == p) orders)

In [128]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

item1 = CartItem product1 2
item2 = CartItem product2 5
item3 = CartItem product3 10

cart1 = ShoppingCart [item1,  item2]
cart2 = ShoppingCart [item2,  item3]


customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Bronze
order1 = Order customer1 cart1 17 Pending
order2 = Order customer2 cart2 20 Processing
order3 = Order customer2 cart2 17 Shipped


searchOrders [ById 2] [order1, order2, order3]
searchOrders [ByTotalPrice 17] [order1, order2, order3]
searchOrders [ByTotalPrice 17, ById 2] [order1, order2, order3]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 20.0, status: Processing
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]